# Import packages

In [248]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, sum, when, col, rand, split, monotonically_increasing_id
from pyspark.sql.types import StringType, FloatType
from sklearn.preprocessing import LabelEncoder
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, PCA, StandardScaler, VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create spark session and load data

In [216]:
spark = SparkSession.builder.appName("analyze").getOrCreate()

file_path = "/home/dotronghiep/Documents/Datasets/Bank_Credit_Default/loan/loan.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [217]:
df.show()

+----+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+----------+----+----+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------

# Dataset's basic infors

In [218]:
num_rows = df.count()
num_columns = len(df.columns)
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

Number of rows: 2260668
Number of columns: 145


In [219]:
print(df.columns)

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_i

# Take useful sample with direct label

In [220]:
# check labels
df.select('loan_status').distinct().show()

+--------------------+
|         loan_status|
+--------------------+
|          Fully Paid|
|     In Grace Period|
|         Charged Off|
|  Late (31-120 days)|
|             Current|
|   Late (16-30 days)|
|             Default|
|            Oct-2015|
|Does not meet the...|
|Does not meet the...|
+--------------------+



In [221]:
# just take 3 labels: 'Fully Paid' is 1, 'Charged Off' and 'Default' are 0
df_useful = df.filter(df.loan_status.isin("Fully Paid", "Charged Off", "Default"))
df_useful_label = df_useful.withColumn("loan_status_binary", when(col("loan_status") == "Fully Paid", 0).otherwise(1)).drop("loan_status")

In [222]:
new_num_rows = df_useful_label.count()
new_num_columns = len(df_useful_label.columns)
print(f"Number of rows: {new_num_rows}")
print(f"Number of columns: {new_num_columns}")

Number of rows: 1303637
Number of columns: 145


In [223]:
# check labels
df_useful_label.select('loan_status_binary').distinct().show()

+------------------+
|loan_status_binary|
+------------------+
|                 1|
|                 0|
+------------------+



# Delete columns have more than 10000 null values

In [224]:
# count the number of missing values in each column
null_counts = df_useful_label.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_useful_label.columns])
null_counts.show()

+-------+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+---------+----------+--------------+----------+-------------------+-------+----------+-------+-------+-------+-----+--------+----------+---+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+

In [225]:
# convert the row to dictionary
null_counts_row = null_counts.collect()[0].asDict()

# find the list have number of null values greater than 10000, else fill with mode
columns_to_drop = []
columns_to_fill = []

for k, v in null_counts_row.items():
    if v > 10000:
        columns_to_drop.append(k)
    else:
        columns_to_fill.append(k)

# drop these columns
df_cleaned = df_useful_label.drop(*columns_to_drop)

# fill missing values with mode

# Điền giá trị thiếu bằng giá trị trung bình tương ứng
for column in columns_to_fill:
    # Tính giá trị mode của cột
    mode_value = df_cleaned.groupBy(column).count().orderBy('count', ascending=False).first()[0]
    
    # Thay thế null bằng giá trị mode
    df_cleaned = df_cleaned.fillna({column: mode_value})

In [226]:
len(df_cleaned.columns)

51

In [227]:
df_cleaned.printSchema()

root
 |-- loan_amnt: integer (nullable = false)
 |-- funded_amnt: integer (nullable = false)
 |-- funded_amnt_inv: double (nullable = false)
 |-- term: string (nullable = false)
 |-- int_rate: double (nullable = false)
 |-- installment: double (nullable = false)
 |-- grade: string (nullable = false)
 |-- sub_grade: string (nullable = false)
 |-- emp_length: string (nullable = false)
 |-- home_ownership: string (nullable = false)
 |-- annual_inc: string (nullable = false)
 |-- verification_status: string (nullable = false)
 |-- issue_d: string (nullable = false)
 |-- pymnt_plan: string (nullable = false)
 |-- purpose: string (nullable = false)
 |-- zip_code: string (nullable = false)
 |-- addr_state: string (nullable = false)
 |-- dti: string (nullable = false)
 |-- delinq_2yrs: string (nullable = false)
 |-- earliest_cr_line: string (nullable = false)
 |-- inq_last_6mths: string (nullable = false)
 |-- open_acc: string (nullable = false)
 |-- pub_rec: string (nullable = false)
 |-- rev

In [228]:
# count the number of missing values in each column of new dataframe
null_counts_cleaned = df_cleaned.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_cleaned.columns])
null_counts_cleaned.show()

+---------+-----------+---------------+----+--------+-----------+-----+---------+----------+--------------+----------+-------------------+-------+----------+-------+--------+----------+---+-----------+----------------+--------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------------+--------------------------+-----------+----------------+--------------+------------------------+-----------+--------------------+---------+-------------+-------------------+--------------------+------------------+
|loan_amnt|funded_amnt|funded_amnt_inv|term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|issue_d|pymnt_plan|purpose|zip_code|addr_state|dti|delinq_2yrs|earliest_cr_line|inq_last_6mths|open_acc|pub_rec|revol_bal|revol_util|total_acc|initial_list_status|

In [229]:
df_cleaned.show()

+---------+-----------+---------------+----------+--------+-----------+-----+---------+----------+--------------+----------+-------------------+--------+----------+------------------+--------+----------+-----+-----------+----------------+--------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+----------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------------+--------------------------+-----------+----------------+--------------+------------------------+-----------+--------------------+---------+-------------+-------------------+--------------------+------------------+
|loan_amnt|funded_amnt|funded_amnt_inv|      term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status| issue_d|pymnt_plan|           purpose|zip_code|addr_state|  dti|delinq_2yrs|earliest_cr_line|inq_last_6mths|open_acc|pub_rec|revol_

# Xóa cột grade, zip_code

Thông tin cột grade được biểu diễn thông qua cột sub_grade

Cột zip_code là ID, không chứa thông tin quan trọng

In [230]:
df_cleaned = df_cleaned.drop('grade', 'zip_code')


In [231]:
len(df_cleaned.columns)

49

# Tách cột tháng-năm

In [232]:
time_columns = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']
for column in time_columns:
    df_cleaned = df_cleaned.withColumn("month_"+ column, split(col(column), "-").getItem(0))
    df_cleaned = df_cleaned.withColumn("year_"+column, split(col(column), "-").getItem(1))
    df_cleaned = df_cleaned.drop(column)

In [233]:
df_cleaned.show()

+---------+-----------+---------------+----------+--------+-----------+---------+----------+--------------+----------+-------------------+----------+------------------+----------+-----+-----------+--------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+----------------+---------------+---------------+-------------+------------------+----------+-----------------------+---------------+--------------------------+-----------+----------------+--------------+------------------------+-----------+--------------------+---------+-------------+-------------------+--------------------+------------------+-------------+------------+----------------------+---------------------+------------------+-----------------+------------------------+-----------------------+
|loan_amnt|funded_amnt|funded_amnt_inv|      term|int_rate|installment|sub_grade|emp_length|home_ownership|annual_inc|verification_status|pymnt_plan|           purpose|addr_state|  dti|delinq

# LabelEncoder 

In [234]:
# list of columns need to encode
text_columns = ['term','sub_grade','emp_length', 'home_ownership', 'verification_status', 
                'pymnt_plan', 'purpose', 'addr_state', 'initial_list_status', 
                'application_type', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag',
                'month_issue_d', 'month_earliest_cr_line', 'month_last_pymnt_d', 'month_last_credit_pull_d']
len(text_columns)

17

In [235]:
for column in text_columns:
    indexer = StringIndexer(inputCol=column, outputCol=f"{column}_index")
    df_cleaned = indexer.fit(df_cleaned).transform(df_cleaned)
    df_cleaned = df_cleaned.drop(column)

In [236]:
df_cleaned.printSchema()

root
 |-- loan_amnt: integer (nullable = false)
 |-- funded_amnt: integer (nullable = false)
 |-- funded_amnt_inv: double (nullable = false)
 |-- int_rate: double (nullable = false)
 |-- installment: double (nullable = false)
 |-- annual_inc: string (nullable = false)
 |-- dti: string (nullable = false)
 |-- delinq_2yrs: string (nullable = false)
 |-- inq_last_6mths: string (nullable = false)
 |-- open_acc: string (nullable = false)
 |-- pub_rec: string (nullable = false)
 |-- revol_bal: string (nullable = false)
 |-- revol_util: string (nullable = false)
 |-- total_acc: string (nullable = false)
 |-- out_prncp: string (nullable = false)
 |-- out_prncp_inv: string (nullable = false)
 |-- total_pymnt: string (nullable = false)
 |-- total_pymnt_inv: string (nullable = false)
 |-- total_rec_prncp: string (nullable = false)
 |-- total_rec_int: string (nullable = false)
 |-- total_rec_late_fee: string (nullable = false)
 |-- recoveries: string (nullable = false)
 |-- collection_recovery_fee

In [237]:
df_cleaned.show()

+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+----------+---------+---------+-------------+----------------+---------------+---------------+-------------+------------------+----------+-----------------------+---------------+--------------------------+-----------+--------------+------------------------+-----------+--------------------+---------+------------------+------------+---------------------+-----------------+-----------------------+----------+---------------+----------------+--------------------+-------------------------+----------------+-------------+----------------+-------------------------+----------------------+-------------------+-------------------------+--------------------------+-------------------+----------------------------+------------------------+------------------------------+
|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|  dti|delinq_2yrs|inq_last_6mths|op

In [238]:
# add id columns from zero to over to df_cleaned
df_cleaned = df_cleaned.withColumn("id", monotonically_increasing_id())

df_encode = df_cleaned

for col_name in df_encode.columns:
    try:
        df_encode = df_encode.withColumn(col_name, col(col_name).cast(FloatType()))
    except ValueError:
        print(f"Column '{col_name}' contains values that cannot be converted to FloatType.")

df_encode = df_encode.dropna()


In [239]:
df_encode.printSchema()

root
 |-- loan_amnt: float (nullable = false)
 |-- funded_amnt: float (nullable = false)
 |-- funded_amnt_inv: float (nullable = false)
 |-- int_rate: float (nullable = false)
 |-- installment: float (nullable = false)
 |-- annual_inc: float (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: float (nullable = true)
 |-- inq_last_6mths: float (nullable = true)
 |-- open_acc: float (nullable = true)
 |-- pub_rec: float (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: float (nullable = true)
 |-- out_prncp: float (nullable = true)
 |-- out_prncp_inv: float (nullable = true)
 |-- total_pymnt: float (nullable = true)
 |-- total_pymnt_inv: float (nullable = true)
 |-- total_rec_prncp: float (nullable = true)
 |-- total_rec_int: float (nullable = true)
 |-- total_rec_late_fee: float (nullable = true)
 |-- recoveries: float (nullable = true)
 |-- collection_recovery_fee: float (nullable = true)
 |-- last_pymnt

In [240]:
# count the number of missing values in each column of new dataframe
null_counts_cleaned = df_encode.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_encode.columns])
null_counts_cleaned.show()

+---------+-----------+---------------+--------+-----------+----------+---+-----------+--------------+--------+-------+---------+----------+---------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+---------------+--------------------------+-----------+--------------+------------------------+-----------+--------------------+---------+------------------+------------+---------------------+-----------------+-----------------------+----------+---------------+----------------+--------------------+-------------------------+----------------+-------------+----------------+-------------------------+----------------------+-------------------+-------------------------+--------------------------+-------------------+----------------------------+------------------------+------------------------------+---+
|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|dti|delinq_2yrs|inq_last_6mths|open_ac

In [241]:
len(df_encode.columns)

54

# Chia train test

In [242]:
class_distribution = df_encode.groupBy('loan_status_binary').count().withColumnRenamed('count', 'class_count')
total_count = df_encode.count()
class_distribution = class_distribution.withColumn('class_ratio', col('class_count') / total_count)

# Hiển thị tỷ lệ mỗi lớp
class_distribution.show()

+------------------+-----------+-------------------+
|loan_status_binary|class_count|        class_ratio|
+------------------+-----------+-------------------+
|               1.0|     261652|0.20074389371894688|
|               0.0|    1041760| 0.7992561062810531|
+------------------+-----------+-------------------+



In [243]:
ratios = class_distribution.select('loan_status_binary', 'class_ratio').rdd.collectAsMap()
print(ratios)

{1.0: 0.20074389371894688, 0.0: 0.7992561062810531}


In [244]:
train_ratio = 0.8

# Tạo DataFrame cho mỗi lớp (label)
df_0 = df_encode.filter(col("loan_status_binary") == 0)
df_1 = df_encode.filter(col("loan_status_binary") == 1)

# Chia dữ liệu cho lớp 0
train_df_0, test_df_0 = df_0.randomSplit([train_ratio, 1 - train_ratio], seed=12345)

# Chia dữ liệu cho lớp 1
train_df_1, test_df_1 = df_1.randomSplit([train_ratio, 1 - train_ratio], seed=12345)

# Kết hợp lại để tạo thành tập huấn luyện và tập kiểm tra cuối cùng
train_df = train_df_0.union(train_df_1)
test_df = test_df_0.union(test_df_1)

# Xáo trộn tập huấn luyện và tập kiểm tra
train_df = train_df.orderBy(rand())
test_df = test_df.orderBy(rand())

# Hiển thị kết quả
print("Train DataFrame Count: ", train_df.count())
print("Test DataFrame Count: ", test_df.count())

train_df.groupBy("loan_status_binary").count().show()
test_df.groupBy("loan_status_binary").count().show()

Train DataFrame Count:  1042125


Test DataFrame Count:  261287


24/06/27 12:33:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:40 WARN RowBasedKeyValueBatch: Calling spill() on

+------------------+------+
|loan_status_binary| count|
+------------------+------+
|               0.0|832879|
|               1.0|209246|
+------------------+------+



24/06/27 12:33:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/27 12:33:55 WARN RowBasedKeyValueBatch: Calling spill() on

+------------------+------+
|loan_status_binary| count|
+------------------+------+
|               0.0|208881|
|               1.0| 52406|
+------------------+------+



# Dùng PCA giảm chiều dữ liệu

In [245]:
# count the number of missing values in each column of new dataframe
a = train_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in train_df.columns])
a.show()

+---------+-----------+---------------+--------+-----------+----------+---+-----------+--------------+--------+-------+---------+----------+---------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+---------------+--------------------------+-----------+--------------+------------------------+-----------+--------------------+---------+------------------+------------+---------------------+-----------------+-----------------------+----------+---------------+----------------+--------------------+-------------------------+----------------+-------------+----------------+-------------------------+----------------------+-------------------+-------------------------+--------------------------+-------------------+----------------------------+------------------------+------------------------------+---+
|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|dti|delinq_2yrs|inq_last_6mths|open_ac

In [246]:
features_cols = train_df.columns
features_cols.remove('loan_status_binary')

# Tạo VectorAssembler để kết hợp các cột đặc trưng thành một vector đặc trưng
assembler = VectorAssembler(inputCols=features_cols, outputCol='features')

# Áp dụng VectorAssembler vào DataFrame ban đầu để biến đổi dữ liệu
train_df_transformed = assembler.transform(train_df)

# Chuẩn hóa các đặc trưng sử dụng StandardScaler
scaler = StandardScaler(inputCol='features', outputCol='scaled_features', withMean=True, withStd=True)
scaler_model = scaler.fit(train_df_transformed)
train_df_scaled = scaler_model.transform(train_df_transformed)

# Áp dụng PCA vào dữ liệu đã chuẩn hóa
pca = PCA(k=10, inputCol='scaled_features', outputCol='pca_features')
pca_model = pca.fit(train_df_scaled)
train_df_pca = pca_model.transform(train_df_scaled).select('loan_status_binary', 'pca_features')

24/06/27 12:35:19 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/06/27 12:35:19 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/06/27 12:35:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [247]:
# Biến đổi VectorAssembler cho test_df
assembler = VectorAssembler(inputCols=features_cols, outputCol='features')
test_df_transformed = assembler.transform(test_df)

# Chuẩn hóa đặc trưng sử dụng StandardScaler
test_df_scaled = scaler_model.transform(test_df_transformed)

# Áp dụng PCA cho test_df
test_df_pca = pca_model.transform(test_df_scaled).select('loan_status_binary', 'pca_features')

# Hiển thị kết quả
test_df_pca.show(5)

+------------------+--------------------+
|loan_status_binary|        pca_features|
+------------------+--------------------+
|               0.0|[3.45812610507071...|
|               0.0|[1.60937960549625...|
|               0.0|[1.05390113723742...|
|               0.0|[-3.1329797672121...|
|               0.0|[0.34484347127299...|
+------------------+--------------------+
only showing top 5 rows



# Train models

In [257]:
# Mô hình Logistic Regression
lr = LogisticRegression(featuresCol='pca_features', labelCol='loan_status_binary')
lr_model = lr.fit(test_df_pca)

# Mô hình Decision Tree
dt = DecisionTreeClassifier(featuresCol='pca_features', labelCol='loan_status_binary')
dt_model = dt.fit(test_df_pca)

# Mô hình Random Forest
rf = RandomForestClassifier(featuresCol='pca_features', labelCol='loan_status_binary')
rf_model = rf.fit(test_df_pca)

In [258]:
# Bước 3: Đánh giá các mô hình trên tập kiểm tra

# Đánh giá mô hình Logistic Regression
lr_predictions = lr_model.transform(test_df_pca)
lr_evaluator = BinaryClassificationEvaluator(labelCol='loan_status_binary')
lr_accuracy = lr_evaluator.evaluate(lr_predictions)

# Đánh giá mô hình Decision Tree
dt_predictions = dt_model.transform(test_df_pca)
dt_evaluator = BinaryClassificationEvaluator(labelCol='loan_status_binary')
dt_accuracy = dt_evaluator.evaluate(dt_predictions)

# Đánh giá mô hình Random Forest
rf_predictions = rf_model.transform(test_df_pca)
rf_evaluator = BinaryClassificationEvaluator(labelCol='loan_status_binary')
rf_accuracy = rf_evaluator.evaluate(rf_predictions)

# Hiển thị kết quả đánh giá
print(f"Logistic Regression - Accuracy: {lr_accuracy}")
print(f"Decision Tree - Accuracy: {dt_accuracy}")
print(f"Random Forest - Accuracy: {rf_accuracy}")

Logistic Regression - Accuracy: 0.9890055217554621
Decision Tree - Accuracy: 0.6790877795072015
Random Forest - Accuracy: 0.9347159441848437


In [ ]:
# print confusion matrix for each model output
lr_predictions.groupBy('loan_status_binary', 'prediction').count().show()
dt_predictions.groupBy('loan_status_binary', 'prediction').count().show()   
rf_predictions.groupBy('loan_status_binary', 'prediction').count().show()
